# 1.RFM theory and usage in Marketing Research field
    

RFM analysis is a method used for analysing existing customer and segmenting them into differenct categories so that marketers could target customers within each specific segment more effectively. It is based on simple theory that people are more likely to respond to a new offer if they had purchsed recently and if they are regular customer who had spent heavyly within the retail business. It is mostly used in direct marketing and normally the data should have each transaction in each row and there may be multiple transactions for each costomer.<br>
RFM stands for the three dimensions:<br>
Recency – How recently did the customer purchase? <br>
Frequency – How often did the customer purchase?<br>
Monetary Value – How much did the customer spend?<br>
In order to classify customers according to their RFM behaviour, each customer is assigned a score based on one of the three aspects.For example,if you use a scale of 1-5 and for the customers with the most recent purchase dates you would rank them score of 1 which means they are the best customer in your theory. Consequently, on the same ranking scheme, you could rank customers who purchase most often as score of 1 and who spend the highest monetary values as score of 1.<br>A mock example is created for reference shown below.

![title](img/rfm.png)

# 2. Dataset used for this tutorial and data cleasing

The dataset will be used for this tutorial is from the link below.<br>
http://archive.ics.uci.edu/ml/datasets/online+retail<br>
This is a transnational data set which contains all the transactions occurring between 01/12/2010 and 09/12/2011 for a UK-based and registered non-store online retail. Since I only use it for illustration usage, I will only filter out Germany customers for this anslysis.



In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import pandas as pd
import numpy as np
import datetime as dt
import sys
sys.version

'3.7.0 (default, Jun 28 2018, 08:04:48) [MSC v.1912 64 bit (AMD64)]'

In [2]:
df=pd.read_excel('Online Retail.xlsx')

df.info()

df.head()

If we divide the data according to countries, we could have a rough idea that UK customers and Germany customers may be the largest two groups for the whole dataset becaue they have relatively large transactions.

In [5]:
df.groupby('Country')['InvoiceNo'].count().sort_values(ascending=False)

Country
United Kingdom          495478
Germany                   9495
France                    8557
EIRE                      8196
Spain                     2533
Netherlands               2371
Belgium                   2069
Switzerland               2002
Portugal                  1519
Australia                 1259
Norway                    1086
Italy                      803
Channel Islands            758
Finland                    695
Cyprus                     622
Sweden                     462
Unspecified                446
Austria                    401
Denmark                    389
Japan                      358
Poland                     341
Israel                     297
USA                        291
Hong Kong                  288
Singapore                  229
Iceland                    182
Canada                     151
Greece                     146
Malta                      127
United Arab Emirates        68
European Community          61
RSA                         58


### I will only use Germany customers for analysis because the data size is relatively small.

In [6]:
Germany=df[df.Country=="Germany"]


Germany.info()

We could see that there are only about 95 unique customers in Germany for the online retail business. 

In [8]:
Germany.nunique()

Germany.describe()

InvoiceNo       603
StockCode      1671
Description    1703
Quantity         68
InvoiceDate     598
UnitPrice       106
CustomerID       95
Country           1
dtype: int64

### Since there should not be any data value less than zero in the Quantify variable,I will remove the negative values. It turn out that there is only 1 customer has the negative value.


In [10]:
Germany = Germany[(Germany['Quantity']>0)]
Germany.shape
Germany.nunique()

(9042, 8)

InvoiceNo       457
StockCode      1665
Description    1697
Quantity         47
InvoiceDate     452
UnitPrice       106
CustomerID       94
Country           1
dtype: int64

### create monetary value 'Sales' 

In [11]:
Germany["Sales"]=Germany["Quantity"]*Germany["UnitPrice"]

Germany["UnitPrice"].describe()

### check purchase date and create cutoff date 

In [13]:
Germany["InvoiceDate"].describe()

Germany["InvoiceDate"]=pd.to_datetime(Germany["InvoiceDate"])

cutoff_date=dt.datetime(2011,12,10)

df1=Germany.copy()

count                    9042
unique                    452
top       2011-01-07 12:28:00
freq                      149
first     2010-12-01 13:04:00
last      2011-12-09 12:16:00
Name: InvoiceDate, dtype: object

# 3. RFM data analysis

In [17]:
rfm=df1.groupby('CustomerID').agg({'InvoiceDate':lambda x: (cutoff_date-x.max()).days,
                                 "InvoiceNo": lambda x: len(x),
                                 "Sales": lambda x: x.sum()})

rfm['InvoiceDate']=rfm['InvoiceDate'].astype(int)

rfm.rename(columns={"InvoiceDate":"Recency",
                   "InvoiceNo":"Frequency",
                   "Sales":"MonetaryValue"},inplace=True)
rfm.sort_values('Recency').tail(10)

The customer with CustomerID=12665 whose last purchase from the online business was nearly one year ago and for the past year the customer only purchased three times with total monetary value worth $63. Comparing with other frequent customers, this customer is definitely far from being our profitable and loyal customer. Let's check the info in detail for curiosity reason.

In [20]:
low_value_customer=df1[df1["CustomerID"]==12665.0]
low_value_customer

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Sales
62995,541517,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2011-01-19 08:34:00,1.25,12665.0,Germany,30.0
62996,541517,POST,POSTAGE,1,2011-01-19 08:34:00,18.00,12665.0,Germany,18.0
62997,541517,21231,SWEETHEART CERAMIC TRINKET BOX,12,2011-01-19 08:34:00,1.25,12665.0,Germany,15.0


## calculate RFM Scores using percentiles

### Since we need to rank each RFM metirc to calculate respective RFM score, the most convinient way to do so is using percentiles. We will split each metric into 5 segments.

In [21]:
quantile=rfm.quantile(q=[.2,.4,.6,.8])
quantile=quantile.to_dict()
quantile

{'Recency': {0.2: 14.600000000000001,
  0.4: 23.400000000000006,
  0.6: 58.8,
  0.8: 129.4},
 'Frequency': {0.2: 20.0,
  0.4: 36.400000000000006,
  0.6: 68.8,
  0.8: 155.20000000000002},
 'MonetaryValue': {0.2: 428.84000000000003,
  0.4: 748.652,
  0.6: 1683.8000000000002,
  0.8: 3885.2880000000005}}

Since we our definition of best customer is customer  with highest monetary value and highest frequency but lowest recency.  We should rank Recency differently compared with the other two variables.

In [22]:
def R_score(x,percentile,d):
    if x<=d[percentile][.2]:
        return str(1)
    elif x<=d[percentile][.4]:
        return str(2)
    elif x<=d[percentile][.6]:
        return str(3)
    elif x<=d[percentile][.8]:
        return str(4)
    else:
        return str(5)
def FM_score(x,percentile,d):
    if x<=d[percentile][.2]:
        return str(5)
    elif x<=d[percentile][.4]:
        return str(4)
    elif x<=d[percentile][.6]:
        return str(3)
    elif x<=d[percentile][.8]:
        return str(2)
    else:
        return str(1)
    

In [23]:
rfm["R_score"]=rfm["Recency"].apply(R_score,args=('Recency',quantile,))
rfm["F_score"]=rfm["Frequency"].apply(FM_score,args=('Frequency',quantile,))
rfm["M_score"]=rfm["MonetaryValue"].apply(FM_score,args=('MonetaryValue',quantile,))
rfm["RFM_score"]=(rfm["R_score"]+rfm["F_score"]+rfm["M_score"]).astype(int)
rfm=rfm.reset_index()

rfm.info()

rfm.sort_values('RFM_score').head(10)

## we could see that only 5 customers out of 94 customers are labeled as '111' which stands for our most valuable and loyal customer.

In [26]:
rfm.sort_values('RFM_score').tail()

,CustomerID,Recency,Frequency,MonetaryValue,R_score,F_score,M_score,RFM_score
53,12622.0,232,11,180.10,5,5,5,555
54,12623.0,276,18,305.10,5,5,5,555
35,12561.0,302,16,238.85,5,5,5,555
84,12738.0,372,11,155.35,5,5,5,555
70,12665.0,324,3,63.00,5,5,5,555


# 4. Summary
By using RFM analysis, we could divide our retail customers into different segments and by doing so we could promote our products more effectively and improve specific KPIs accordingly.